# LangChain: Evaluation¶
### Outline:
* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [3]:
import os
import getpass
import openai
import time
import markdown
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pickle
import json
from IPython.display import display, Markdown

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:········


# Create our QandA application

In [4]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

#PDF directory loader
from langchain.document_loaders import PyPDFDirectoryLoader

In [7]:

start_time = time.time()

pdf_folder_path = "/Users/markc/Hydrogen_mini/"
loader = PyPDFDirectoryLoader(pdf_folder_path)
data = loader.load_and_split()


end_time = time.time()
run_time = end_time-start_time
print(run_time)

# For 3900 pages...
#loader.load() takes 137 sec. 
#loader.load_and_split() takes 129 sec.6800 pgs -> 300s

78.30924820899963


In [8]:
print(type(data))
data[111]
len(data)

<class 'list'>


1462

In [9]:
df = pd.DataFrame(data)
df.astype(str)
df.rename(columns={0: 'input', 1: 'source'},inplace=True)
df.size

2924

In [10]:
# Convert columns to string so to_parquet will work
df[df.columns[0]]=df[df.columns[0]].astype(str)
df[df.columns[1]]=df[df.columns[1]].astype(str)
print(df.head(10))

                                               input  \
0  ('page_content', 'NREL is a national laborator...   
1  ('page_content', 'NREL is a national laborator...   
2  ('page_content', 'NOTICE  \nThis work was auth...   
3  ('page_content', 'iii \nThis report is availab...   
4  ('page_content', 'iv \nThis report is availabl...   
5  ('page_content', 'v \nThis report is available...   
6  ('page_content', 'vi \nThis report is availabl...   
7  ('page_content', '4 While the combined technic...   
8  ('page_content', 'vii \nThis report is availab...   
9  ('page_content', 'viii \nThis report is availa...   

                                              source  
0  ('metadata', {'source': '\\Users\\markc\\Hydro...  
1  ('metadata', {'source': '\\Users\\markc\\Hydro...  
2  ('metadata', {'source': '\\Users\\markc\\Hydro...  
3  ('metadata', {'source': '\\Users\\markc\\Hydro...  
4  ('metadata', {'source': '\\Users\\markc\\Hydro...  
5  ('metadata', {'source': '\\Users\\markc\\Hydro... 

In [11]:
# Jenny from Estuary notes Parquet is "not easy to work with" so will try JSON format. 
# df.to_parquet('data.parquet')

#convert to dataframe to JSON file
json_data = df.to_json(orient='records')


In [12]:
with open('data.json', 'w') as file:
    file.write(json_data)

In [13]:
# Efficient vectorstor method?
# Setting vector indices for data
start_time = time.time()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

end_time = time.time()
run_time = end_time-start_time
print(run_time)

95.45564579963684


In [14]:
# Why does chain_type not accept map_reduce?
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    return_source_documents=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

# Coming up with test datapoints

In [15]:
data[410]

Document(page_content='Global Hydrogen Review 2022   \nPAGE | 98  Hydrogen  production  \n \nOffshore wind can become a cost -effective option for hydrogen production at high full load \nhours\nHydrogen production costs from offshore wind in the Net Zero Emissions Scenario, 2030 (left figure)  \nSupply cost curves for hydrogen production with offshore wind electricity generation, 2030 and 2050 (right figure)  \n \n \nIEA. All rights reserved.  \nThis map is without prejudice to the status of or sovereignty over any territory, to the delim itation of international frontiers and boundaries and to the name of any territory, city or area.  \n \nNote: Only sites with an annual average capacity factor above 20% are considered in the supply cost curves in the right figur e. \nSource: Based on hourly wind data from Copernicus Climate Change Service, exclusive economic zones from Marine Regions  and protected areas from the World Database on \nProtected Areas .', metadata={'source': '\\Users\\m

In [16]:
len(data)

1462

# LLM-Generated examples

In [78]:
# Four boxes below generate Q&A pair to evaluate model
from langchain.evaluation.qa import QAGenerateChain

# Pass in OpenIA language model to interact with chain
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [79]:
import langchain
langchain.debug = True

In [80]:
# Get back dictionary of question/answer pairs to evaluate
#
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:9]]
)

[chain/start] [1:chain:QAGenerateChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:QAGenerateChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\npage_content='NREL is a national laboratory of the U.S. Department of Energy  \\nOffice of Energy Efficiency & Renewable Energy  \\nOperated by the Alliance for Sustainable Energy, LLC  \\nThis report is available at no cost from the National Renewable Energy  \\nLaboratory (NREL) at www.nrel.gov/publications.   \\n Contract No. DE -AC36 -08GO28308  \\n  \\nTechnical Report  \\nNREL/TP -5400- 77198  \\nJuly 2020  Resourc

[llm/end] [1:chain:QAGenerateChain > 2:llm:ChatOpenAI] [9.94s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "QUESTION: What is the organization that operates the National Renewable Energy Laboratory (NREL)?\n\nANSWER: The Alliance for Sustainable Energy, LLC operates the National Renewable Energy Laboratory (NREL).",
        "generation_info": null,
        "message": {
          "content": "QUESTION: What is the organization that operates the National Renewable Energy Laboratory (NREL)?\n\nANSWER: The Alliance for Sustainable Energy, LLC operates the National Renewable Energy Laboratory (NREL).",
          "additional_kwargs": {},
          "example": false
        }
      }
    ],
    [
      {
        "text": "QUESTION: What organization operates the National Renewable Energy Laboratory (NREL)?\n\nANSWER: The Alliance for Sustainable Energy, LLC operates the National Renewable Energy Laboratory (NREL).",
        "generation_info": null,
        "me

ValueError: Could not parse output: QUESTION: What is the organization that operates the National Renewable Energy Laboratory (NREL)?

ANSWER: The Alliance for Sustainable Energy, LLC operates the National Renewable Energy Laboratory (NREL).

In [11]:
# See example
new_examples[9]

examples = []
examples += new_examples

qa.run(examples[9]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


"Since the publication of the 2021 Annual Evaluation, there have been a total of 8 new Open-Retail hydrogen fueling stations added to California's network, 6 of which opened in 2022."



> Entering new RetrievalQA chain...

> Finished chain.


"Since the 2021 Annual Evaluation was published, a total of 8 new Open-Retail hydrogen fueling stations have been added to California's network, 6 of which opened in 2022."

# Manual Evaluation

In [12]:
import langchain
langchain.debug = True

In [13]:
qa.run(examples[4]["query"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Where was the source of the document located?"
}
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "Where was the source of the document located?",
  "context": "by-product sources.<<<<>>>>>information. Web posting. Summaries. \nVerDate Sep 11 2014 12:25 Dec 28, 2021 Jkt 029139 PO 00058 Frm 00314 Fmt 6580 Sfmt 6581 E:\\PUBLAW\\PUBL058.117 PUBL058whamilton on LAPJF8D0R2PROD with PUBLAW<<<<>>>>>note. Records. Estimate. \nVerDate Sep 11 2014 12:25 Dec 28, 2021 Jkt 029139 PO 00058 Frm 00345 Fmt 6580 Sfmt 6581 E:\\PUBLAW\\PUBL058.117 PUBL058whamilton on LAPJF8D0R2PROD with PUBLAW<<<<>>>>>1992 -UC-38; Pacific Northwest Laborator y: Richland, W A; 1976 ."
}
[llm/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain

'The source of the document is not specified.'

In [86]:
langchain.debug = False

In [19]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [20]:
# QAEvalChain evaluates question answer pairs
from langchain.evaluation.qa import QAEvalChain

In [21]:
# Create above chain with language model. LLM will help do evaluation
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [22]:
# Get back graded outputs
graded_outputs = eval_chain.evaluate(examples, predictions)

NameError: name 'examples' is not defined

In [23]:
# All below are output by the language mdoel
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: What is the title of the report and what legislation is it pursuant to?
Real Answer: The title of the report is "2022 Annual Evaluation of Fuel Cell Electric Vehicle Deployment and Hydrogen Fuel Station Network Development" and it is pursuant to Assembly Bill 8, also known as Perea, Chapter 401, Statutes of 2013.
Predicted Answer: The context does not provide a clear answer to this question. However, it mentions several reports that the Secretary and Chiefs are required to submit to Congress, including a report with recommendations to Congress relating to the Program, a report describing methods, and additional reports for fiscal years 2022 and 2023. The legislation that these reports are pursuant to is also not specified in the context.
Predicted Grade: INCORRECT

Example 1:
Question: Who reviewed and approved the report for publication?
Real Answer: The staff of the California Air Resources Board (CARB) reviewed and approved the report for publication.
Predicted 

## Key Questions/Concerns for Dev

In [69]:
# What to do if predicted answers incorrect?
# QAEvalChain spit out the same question multiple times
# Try to access the UI that tracks what is going on 
#  under the hood (from langchain plus)
# --generate flywheel of datapoints to learn from!!!!



### Try to query this new vectorstor database

In [17]:
query = "What system design results in the lowest hydrogen breakeven cost? \
What is that cost in USD/kg? Is this with or without subsidies? Use markdown to describe."

# Decent answer. High level, but limited by propmt.

In [18]:
query2 = "What are the major challenges to deploying hydrogen and how can they become overcome?. \
Use bulleted markdown list to describe."

# Decent answer. Relatively high level. Same as ChatGPT.

In [19]:
query3 = "I wish to deploy hydrogen production in California. What technology should I use, where shoult it be placed,  \
what commercial models are suggested, and what federal or state incentives are applicable? Use markdown to describe."

# Not a good answer. Very generic.

In [20]:
query4 = "What federal incentives exist for hydrogen production? \
Use markdown to describe."

# Not specific enough answer. ChatGPT has better answer.

In [21]:
query5 = "What are ideal conditions for hydrogen in use as long duration storage? \
Use markdown to describe in fewer than 800 words in bulleted list and list hydrogen's top competitors."

# Good answer.

In [22]:
query6 = "You are an investor. What are ideal conditions and locations for hydrogen for use as long duration storage? \
Use markdown to describe in fewer than 800 words."

In [23]:
query7 = "Please describe how a PEM electrolyzer, alkaline electrolyzer, and solid oxide electrolyzer work. What are advantages and disadvantages for each? \
What is the average levelized cost of hydrogen production (in USD/kg) for each? Use markdown to describe in fewer than 1000 words."

# Good answer

In [24]:
# Trying to gauge breadth and depth of model's knowledge

query8 = "Please tell me the most cost-effective, scalable ways to decarbonize with hydrogen and \
the applications which hydrogen is most competitive in  decarbonizing. Do this in markdown in a bulleted list."

# Decent answer. Lacks a bit of detail.

In [25]:
query9 = "Please list the highest TRL hydrogen production methods and the range of their levelized cost of production in USD/kg. \
Do this in markdown in a bulleted list."

#Good response. Same as ChatGPT

In [26]:
query10 = "What clean energy technologies are most competitive with hydrogen? \
What technologies are complementary to hydrogen? Display results in markdown in a table."

#Poor response - probably due to knowledge base

In [27]:
query11 = "What are the main hydrogen provisions in the Clean Hydrogen Act? \
Create a bulleted list with short description. Display results in markdown in a table."

#Poor response - probably due to knowledge base

### Added questions with larger database

In [28]:
query12 = "Describe in details what the optimal path is to decarbonize California's grid. \
provide detail in no more than 1000 words and use a bulleted list in markdown to describe. \
What are the main renewable technologies California plans to use?"

# Decent answer, but very high level. Might be a result of information

In [29]:
query13 = "What are the major challenges in using hydrogen in existing existing oil & gas infrastructure? \
 Provide a response in fewer than 1500 words using markdown."

# Great answer. Slightly better than ChatGPT.

In [30]:
query14 = "Describe hydrogen's main physical characteristics such as flammability range, energy density, ignition energy, etc. \
Include the top 10 most cited physical characteristics and output in markdwon table."

# Wow! Great answer. Same as ChatGPT.

In [31]:
query15 = "How does the cost of producing hydrogen from electrolysis compare with SMR w/CCS? \
Are there any particular regions in the US where electrolytic hydrogen is more cost effective? What makes that so? \
Provide an answer in bulleted list form in markdown. "

# Not a great answer. I feel prompt may be too generic.

In [32]:
query16 = "How should I think about deploying cost-effective electrolytic hydrogen? What are the major considerations? \
Provide an answer in bulleted list form in markdown. "

# Good answer. Same as ChatGPT

In [33]:
query17 = "What are some ways to invest in the hydrogen market? Briefly describe each opportunity \
Provide an answer in bulleted list form in markdown in fewer than 1000 words. "

# Good answer. Nearly same as ChatGPT.

In [34]:
query18 = "Describe how CCS and CCUS work. What is the approximate cost of each technology? How do they pair with  \
hydrogen technologies? Provide an answer in bulleted list form in markdown in fewer than 1000 words. "

# Good answer

In [35]:
query19 = "Describe the key differences between SMR and ATR. What are they? Which process is more efficient? \
Which technology is more cost effective? Provide an answer in bulleted list form in markdown in fewer than 1000 words. "

# Good answer. About same as ChatGPT.

In [36]:
query20 = "What carbon capture technologies should I consider for a SMR unit? How about for an ATR unit? \
What are the advantages and disadvantages for each carbon capture technology when paired with SMR or ATR? \
Provide an answer in bulleted list form in markdown in fewer than 2000 words. "

# Good answer

In [37]:
query21 = "How much does it cost to construct a natural gas pipeline? How about a hydrogen pipeline? \
What are the major differenences between the two? \
Provide an answer in bulleted list in markdown in fewer than 2000 words. "

# very mediocre answer. Need more depth here in this topic.

In [38]:
query22 = "How does liquefaction of hydrogen work? What is the cost of liquefaction at various scales? \
In what scenarios is gaseous delivery of hydrogen less economic the delivery of liquid hydrogen? \
Provide an answer in a bulleted list form in markdown in fewer than 1500 words. "

# Pretty good 1st order answer

In [39]:
query23 = "Provide an overview of the various hydrogen carriers that could be used to transport hydrogen. \
How do the costs of each carrier compare? In what scnearios is one carrier favored over others? \
Provide an answer in a table form in markdown in fewer than 1500 words. "

# Pretty good 1st order answer. About the same as ChatGPT

In [40]:
query24 = "Provide an overview of how natural hydrogen is produced. What are the processes? Where can it be found? \
Provide an answer in a table form in markdown in fewer than 1500 words. "

# Pretty good 1st order answer

In [41]:
query25 = "Provide an in-depth review of how natural hydrogen occurs. What are the different processes? Where can it be found? \
Provide an answer in a table form in markdown in fewer than 3000 words. "

# Not a great answer. Why is this? ChatGPT's answer is far worse.

In [43]:
start_time = time.time()
response = index.query(query19, llm=llm)
#response = qa_stuff.run(query)

run_time = time.time()-start_time
print(run_time)

display(Markdown(response))

6.103835105895996


- SMR (Steam Methane Reforming) and ATR (Autothermal Reforming) are both technologies used for hydrogen production from natural gas.
- In SMR, natural gas is combusted with a portion of it being used as fuel for the process, while the rest is split into hydrogen and concentrated CO2. A diluted CO2 stream is produced in this process.
- In ATR, the required heat is produced within the reformer itself, resulting in all the CO2 being produced inside the reactor. This allows for higher CO2 recovery rates compared to SMR.
- ATR also allows for the capture of emissions at a lower cost than SMR because the emissions are more concentrated.
- SMR is the most widespread technology for large-scale hydrogen production due to its favorable economics and the large number of SMR units in operation.
- CCUS (Carbon Capture, Utilization, and Storage) can be applied to both SMR and ATR hydrogen production, leading to a reduction in carbon emissions of up to 90%.
- Several SMR-CCUS plants are already operational, producing low-carbon hydrogen.
- A number of studies have shown that the costs of SMR with capture rates exceeding 90% are higher than that of a comparable ATR system.
- A large share of global ammonia and methanol production combines SMR with ATR technology.
- The announced HyNet and H21 projects in the United Kingdom plan to use ATR with CCUS instead of SMR.
- The efficiency of SMR and ATR in terms of hydrogen production is not explicitly mentioned in the given context.
- The cost effectiveness of SMR and ATR is not explicitly mentioned in the given context.

### Section to test out returning citations

In [44]:
from langchain.chains import RetrievalQAWithSourcesChain

In [45]:
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, 
                                chain_type="stuff",
                                retriever=index.vectorstore.as_retriever(),
                                return_source_documents=False)

In [46]:
qa_with_sources(query15)

{'question': 'How does the cost of producing hydrogen from electrolysis compare with SMR w/CCS? Are there any particular regions in the US where electrolytic hydrogen is more cost effective? What makes that so? Provide an answer in bulleted list form in markdown. ',
 'answer': '- The production cost of grid-based electrolytic hydrogen in the United States is already cost-effective today. (Source: \\Users\\markc\\Hydrogen_mini\\Guerra et al - 2019 - Cost Competitiveness of Electrolytic Hydrogen.pdf)\n- CO2-free renewables, such as renewable electricity, could be among the cheapest hydrogen sources, but only in very particular situations. (Source: \\Users\\markc\\Hydrogen_mini\\Hydrogen A renewable energy perspective.pdf)\n- The cost of producing hydrogen from electrolysis varies by region and depends on factors such as local regulations, financial aspects, and the price of renewable power and fossil fuels. (Source: \\Users\\markc\\Hydrogen_mini\\Hydrogen A renewable energy perspective.p

####  Key Question/Concerns for Dev

In [ ]:
# Getting unclear answers when combining all four reports
# Does not seem to be correctly parsing through the data
# Can it not read tables?
# Why can't it generate more accurate answers?

# QAGenerateChain.from_llm -> generates same questions. How to get higher fidelity questions?

### List of topics to add

* Transportation economics
* Natural hydrogen